Music Creation using LSTM

**Installing Libraries**

h5py will be required later while training the model in order to store trained model weights after every epoch

In [0]:
# h5py will be required later while training the model in
pip install h5py

In [0]:
# Importing required classes
from pydub import AudioSegment
import os
import glob
import numpy as np
from music21 import converter, instrument, note, chord

# importing Keras libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

The dataset is uploaded from the system into the midi_songs directory. This directory contains 92 .mid extension files.

In [0]:
#os.mkdir('midi_songs')
#%cd ../
os.getcwd()

In [0]:
# Making Notes list of all notes and chords using file path
notes = []

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)

    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))  

len(notes)              

In [0]:
seq_length = 100
# sorting all notes and chords pitches
pitch_names = sorted(set(item for item in notes))

# total unique notes/combination_of_notes in the dataset
len(pitch_names)

# Creating note_to_int dictionary
note_to_indx = dict((note,i) for i,note in enumerate(pitch_names))

In [0]:
# Creating inputs and corresponding outputs for LSTM model
input_seq = []
output_seq = []

for i in range(0,len(notes)-seq_length,1):
  seq_in = notes[i:i+seq_length]
  seq_out = notes[i+seq_length]
  input_seq.append([note_to_indx[key] for key in seq_in])
  output_seq.append(note_to_indx[seq_out])

In [0]:
n_patterns = len(input_seq)
# reshape the input into a format compatible with LSTM layers
input_seq = np.reshape(input_seq, (n_patterns, seq_length, 1))
input_seq = input_seq/len(pitch_names)
output_seq = np_utils.to_categorical(output_seq)
input_seq.shape

Defining the model using keras Sequential class. After which it is trained and the weights are saved.

In [0]:
model = Sequential()

model.add(LSTM(256,input_shape=(input_seq.shape[1],input_seq.shape[2]),return_sequences=True))
model.add(Dropout(0.13))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(pitch_names)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(input_seq, output_seq, epochs=40, batch_size=100, callbacks=callbacks_list)

Building another structure but htis time instead of training loading it with weights obtained during traing 

In [0]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(input_seq.shape[1], input_seq.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(358))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights('/content/weights-improvement-40-0.5465-bigger.hdf5')

Making predictions

In [0]:
# prediction
start = np.random.randint(0, len(input_seq)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitch_names))
pattern = input_seq[start]
prediction_output = []
pattern = np.array(pattern)

# generate 500 notes
for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(pitch_names))

        prediction = model.predict(prediction_input, verbose=0)
        prediction = np.reshape(prediction,(358,1))
        index = np.argmax(prediction)
        result = int_to_note[index]
        #making index size as that of pattern to concat.
        index = [[index]]
        prediction_output.append(result)
        pattern = np.concatenate((pattern,index))
        pattern = pattern[1:len(pattern)+1]
        # see predicted sequence of notes
        print(result)

obtaining notes

In [0]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5


Creating the MIDI file 

the file can later be converted to .mp3 format from [here](https://www.zamzar.com/) for listening your predicted music.


In [206]:
from music21 import stream
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'